In [38]:
import matplotlib as mpl
from matplotlib.widgets import RectangleSelector

mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import cv2
import open3d
from intrinsics import Intrinsics
from skimage.restoration import inpaint
from tqdm.notebook import tqdm
from utils import pcd_to_depth_map, pcd_to_volume
import math
import scipy
from scipy.spatial.transform import Rotation

In [39]:
fps = 30
test_name = "test2"
scale = 0.00025
path = glob.glob(os.path.join("techmed_test", test_name, "depth", "*.png"))[500]

depth = np.asanyarray(open3d.io.read_image(path)) * scale
depth = open3d.geometry.Image(depth.astype(np.float32))

plt.imshow(depth)
plt.show()

In [40]:
points = []

def mouse_event(event):
    global points
    global rgb_copy

    points.append([int(event.xdata), int(event.ydata)])

    if len(points) == 3:
        plt.close()

fig = plt.figure(figsize=(10, 10))
cid = fig.canvas.mpl_connect('button_press_event', mouse_event)
# plt.imshow(cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB))
plt.imshow(depth)
plt.show()

# rgb_copy = rgb.copy()
depth_copy = np.asarray(depth).copy()
for p in points:
    # marked_image = cv2.circle(rgb_copy, p, 3, (0, 255, 0), 2)
    marked_image = cv2.circle(depth_copy, p, 3, (0, 255, 0), 2)

fig = plt.figure(figsize=(10, 10))
# plt.imshow(cv2.cvtColor(rgb_copy, cv2.COLOR_BGR2RGB))
plt.imshow(marked_image)
plt.show()

In [41]:
fig, ax = plt.subplots()
ax.imshow(depth, cmap="gray")
roi = None

def line_select_callback(eclick, erelease):
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    global roi
    roi = [[int(x1), int(y1)], [int(x2), int(y2)]]
    plt.close()

rs = RectangleSelector(ax, line_select_callback,
                       useblit=False, button=[1],
                       minspanx=5, minspany=5, spancoords='pixels',
                       interactive=True)


plt.connect("key_press_event", rs)
plt.show()

In [42]:
roi

In [43]:
npdepth = np.asarray(depth)
temp_mask = np.zeros(npdepth.shape)
mask = np.zeros(npdepth.shape)

temp_mask[npdepth == 0] = 1

mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]] = temp_mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]]

plt.imshow(mask)
plt.show()
# mask.shape

TypeError: 'NoneType' object is not subscriptable

In [ ]:
intr = Intrinsics(width=640, height=480)

In [ ]:
point_mask = np.zeros(npdepth.shape)

for p in points:
    point_mask[p[1], p[0]] = npdepth[p[1], p[0]]

markers_o3d = open3d.geometry.Image(point_mask.astype(np.float32))
intr = Intrinsics(width=640, height=480)

m_pcd = open3d.geometry.PointCloud.create_from_depth_image(markers_o3d, intrinsic=intr.get_intrinsics())
m_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
m_pcd.paint_uniform_color([1.0, 0, 0])

points3d = np.asarray(m_pcd.points).copy()

u_1 = points3d[1] - points3d[2]
u_2 = points3d[0] - points3d[2]

n = np.cross(u_1, u_2)
R = Rotation.align_vectors([0, 1, 0], n)[0].as_matrix()

In [ ]:
m_pcd.translate(-points3d[2])
m_pcd.rotate(R, center=(0, 0, 0))

In [ ]:
npdepth = inpaint.inpaint_biharmonic(npdepth, mask)
    
depth = open3d.geometry.Image(npdepth.astype(np.float32))

pcd = open3d.geometry.PointCloud.create_from_depth_image(depth=depth, intrinsic=intr.get_intrinsics())
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
pcd.translate(-points3d[2])
pcd.rotate(R, center=(0, 0, 0))

In [ ]:
axes = open3d.geometry.TriangleMesh.create_coordinate_frame()
open3d.visualization.draw_geometries([pcd, axes])